In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from ray import tune
# from ray.rllib.agents.ppo import PPOTrainer

# Define a simple model for demonstration (e.g., LeNet for MNIST)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training function with model pruning
def train_model(config):
    pruning_percentage = config["pruning_percentage"]
    learning_rate = config["learning_rate"]
    retraining_epochs = config["retraining_epochs"]

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_data = datasets.MNIST(root="data", train=True, download=True, transform=transform)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    model = SimpleCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            torch.nn.utils.prune.l1_unstructured(module, name='weight', amount=pruning_percentage)

    for epoch in range(retraining_epochs):
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    accuracy = evaluate_model(model)
    compression_ratio = calculate_compression_ratio(model)

    tune.report(accuracy=accuracy, compression_ratio=compression_ratio, reward=accuracy - compression_ratio * 0.1)

def evaluate_model(model):
    test_data = datasets.MNIST(root="data", train=False, download=True, transform=transforms.ToTensor())
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return correct / total

def calculate_compression_ratio(model):
    total_params = sum(p.numel() for p in model.parameters())
    pruned_params = sum(p.numel() for p in model.parameters() if hasattr(p, "weight_mask"))
    return (total_params - pruned_params) / total_params

# RLlib-based optimization
def run_rllib_search():
    # Define the search space
    search_space = {
        "pruning_percentage": tune.uniform(0.1, 0.8),
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "retraining_epochs": tune.randint(1, 10),
    }

    analysis = tune.run(
        train_model,
        config=search_space,
        metric="reward",
        mode="max",
        num_samples=20,
    )

    print("Best hyperparameters found were: ", analysis.best_config)

if __name__ == "__main__":
    run_rllib_search()


2024-11-29 18:04:05,785	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-11-29 18:05:17,072	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-11-29 18:05:17,078	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/sidharrthnagappan/ray_results/train_model_2024-11-29_18-04-05' in 0.0051s.
2024-11-29 18:05:17,100	INFO tune.py:1041 -- Total run time: 71.31 seconds (71.22 seconds for the tuning loop).
2024-11-29 18:05:17,100	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-11-29 18:05:17,106	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 20 trial(s):
- train_model_52b4a_00000: FileNotFoundError('Could not fetch metrics for train_model_52b4a_00000: both result.json and progress.csv were not found at /Users/sidharrthnagappan/ray_resu

Best hyperparameters found were:  None


In [18]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env


# Define a simple CNN model for compression
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Define the custom RL environment
class CompressionEnv(gym.Env):  # Inherit from gymnasium.Env
    metadata = {"render_modes": ["human"], "render_fps": 4}

    def __init__(self, config):
        super(CompressionEnv, self).__init__()

        # Define action space: [pruning percentage, learning rate]
        self.action_space = spaces.Box(
            low=np.array([0.1, 1e-4]), high=np.array([0.8, 1e-2]), dtype=np.float32
        )

        # Define observation space: can include state info (e.g., current accuracy, size)
        self.observation_space = spaces.Box(low=0, high=1, shape=(3,), dtype=np.float32)

        # Load dataset
        transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
        )
        self.train_data = datasets.MNIST(
            root="data", train=True, download=True, transform=transform
        )
        self.test_data = datasets.MNIST(
            root="data", train=False, download=True, transform=transform
        )
        self.train_loader = DataLoader(self.train_data, batch_size=64, shuffle=True)
        self.test_loader = DataLoader(self.test_data, batch_size=64, shuffle=False)

        # Initialize model and other variables
        self.model = SimpleCNN()
        self.initial_params = sum(p.numel() for p in self.model.parameters())
        self.state = np.array(
            [0, 0, 0]
        )  # Placeholder state: accuracy, compression ratio, steps
        self.episode_length = 10

    def reset(self, seed=None, options=None):
        # Reset the environment for a new episode
        super().reset(seed=seed)
        self.model = SimpleCNN()  # Reinitialize the model
        self.state = np.array([0, 0, 0])  # Reset state
        return self.state, {}

    def step(self, action):
        pruning_percentage, learning_rate = action

        # Apply pruning
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                torch.nn.utils.prune.l1_unstructured(
                    module, name="weight", amount=pruning_percentage
                )

        # Retrain the model
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        self.model.train()
        for epoch in range(1):  # Train for 1 epoch per step
            for data, target in self.train_loader:
                optimizer.zero_grad()
                output = self.model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

        # Evaluate the model
        accuracy = self.evaluate_model()
        compression_ratio = self.calculate_compression_ratio()

        # Update state
        self.state = np.array([accuracy, compression_ratio, self.state[2] + 1])

        # Reward: prioritize accuracy but incentivize compression
        reward = accuracy - 0.1 * compression_ratio

        done = self.state[2] >= self.episode_length
        return self.state, reward, done, {}

    def evaluate_model(self):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in self.test_loader:
                output = self.model(data)
                _, predicted = torch.max(output, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        print(f"Accuracy: {correct / total}")
        return correct / total

    def calculate_compression_ratio(self):
        pruned_params = sum(
            p.numel() for p in self.model.parameters() if hasattr(p, "weight_mask")
        )
        return (self.initial_params - pruned_params) / self.initial_params


# Register the environment with RLlib
def create_env(config):
    return CompressionEnv(config)


register_env("CompressionEnv", create_env)

# RLlib Training Configuration
if __name__ == "__main__":
    config = (
        PPOConfig()
        .environment(env="CompressionEnv")
        .framework("torch")
        .api_stack(
            enable_rl_module_and_learner=True, enable_env_runner_and_connector_v2=True
        )
        .env_runners(
            num_env_runners=5, sample_timeout_s=None
        )  # Set the number of environment runners
        .training(model={"fcnet_hiddens": [128, 128]})
    )

    # Build the PPO algorithm
    algo = config.build()

    # Train the algorithm
    for i in range(50):  # Run 50 iterations
        result = algo.train()
        print(f"Iteration {i}: reward_mean={result}")

    # Save the trained policy
    algo.save("./ppo_compression_policy")

2024-11-29 17:59:00,073	WARNING algorithm_config.py:4484 -- You configured a custom `model` config (probably through calling config.training(model=..), whereas your config uses the new API stack! In order to switch off the new API stack, set in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. If you DO want to use the new API stack, configure your model, instead, through: `config.rl_module(model_config={..})`.
2024-11-29 17:59:00,074	WARNING ppo.py:305 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
/Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:567: Ra

Iteration 0: reward_mean={'timers': {'training_iteration_time_sec': 61.84202218055725, 'restore_workers_time_sec': 6.198883056640625e-06, 'training_step_time_sec': 61.84197497367859}, 'fault_tolerance': {'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}, 'done': False, 'training_iteration': 1, 'trial_id': 'default', 'date': '2024-11-29_18-00-24', 'timestamp': 1732903224, 'time_this_iter_s': 61.84218978881836, 'time_total_s': 61.84218978881836, 'pid': 33553, 'hostname': 'user-116-99.svr-vpn-0.vpn.cl.cam.ac.uk', 'node_ip': '127.0.0.1', 'config': {'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'placement_strategy': 'PACK', 'num_gpus': 0, '_fake_gpus': False, 'num_cpus_for_main_process': 1, 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU'

2024-11-29 18:00:26,061	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().


Iteration 1: reward_mean={'timers': {'training_iteration_time_sec': 31.919841051101685, 'restore_workers_time_sec': 0.9971414804458618, 'training_step_time_sec': 30.92265748977661}, 'fault_tolerance': {'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}, 'done': False, 'training_iteration': 2, 'trial_id': 'default', 'date': '2024-11-29_18-00-26', 'timestamp': 1732903226, 'time_this_iter_s': 1.997734785079956, 'time_total_s': 63.839924573898315, 'pid': 33553, 'hostname': 'user-116-99.svr-vpn-0.vpn.cl.cam.ac.uk', 'node_ip': '127.0.0.1', 'config': {'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'placement_strategy': 'PACK', 'num_gpus': 0, '_fake_gpus': False, 'num_cpus_for_main_process': 1, 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU':

2024-11-29 18:00:28,059	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
2024-11-29 18:00:28,074	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 1 back into service.
2024-11-29 18:00:28,075	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 2 back into service.
2024-11-29 18:00:28,075	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 3 back into service.
2024-11-29 18:00:28,076	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 4 back into service.
2024-11-29 18:00:28,078	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 5 back into service.


...

2024-11-29 18:01:30,111	ERROR actor_manager.py:804 -- Ray error (The actor 83c6794e5a879b19dd98ca4d01000000 is unavailable: The actor is temporarily unavailable: UnexpectedSystemExit: Worker exits with an exit code 1.. The task may or maynot have been executed on the actor.), taking actor 1 out of service.
2024-11-29 18:01:30,112	ERROR actor_manager.py:804 -- Ray error (The actor 43443ceacd277efbf370d9bd01000000 is unavailable: The actor is temporarily unavailable: UnexpectedSystemExit: Worker exits with an exit code 1.. The task may or maynot have been executed on the actor.), taking actor 2 out of service.
2024-11-29 18:01:30,112	ERROR actor_manager.py:804 -- Ray error (The actor 0900c848171ed111d550a2a201000000 is unavailable: The actor is temporarily unavailable: UnexpectedSystemExit: Worker exits with an exit code 1.. The task may or maynot have been executed on the actor.), taking actor 3 out of service.
2024-11-29 18:01:30,113	ERROR actor_manager.py:804 -- Ray error (The actor 7

Iteration 3: reward_mean={'timers': {'training_iteration_time_sec': 31.96889078617096, 'restore_workers_time_sec': 1.0025712251663208, 'training_step_time_sec': 30.966251254081726}, 'fault_tolerance': {'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 5}, 'done': False, 'training_iteration': 4, 'trial_id': 'default', 'date': '2024-11-29_18-01-30', 'timestamp': 1732903290, 'time_this_iter_s': 62.049379110336304, 'time_total_s': 127.87641668319702, 'pid': 33553, 'hostname': 'user-116-99.svr-vpn-0.vpn.cl.cam.ac.uk', 'node_ip': '127.0.0.1', 'config': {'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'placement_strategy': 'PACK', 'num_gpus': 0, '_fake_gpus': False, 'num_cpus_for_main_process': 1, 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU'

2024-11-29 18:01:32,112	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
2024-11-29 18:01:32,118	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 1 back into service.
2024-11-29 18:01:32,119	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 2 back into service.
2024-11-29 18:01:32,120	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 3 back into service.
2024-11-29 18:01:32,120	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 4 back into service.
2024-11-29 18:01:32,120	WARNING actor_manager.py:817 -- Bringing previously unhealthy, now-healthy actor 5 back into service.


Iteration 4: reward_mean={'timers': {'training_iteration_time_sec': 25.97334804534912, 'restore_workers_time_sec': 1.2001412391662598, 'training_step_time_sec': 24.773144006729126}, 'fault_tolerance': {'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 5}, 'done': False, 'training_iteration': 5, 'trial_id': 'default', 'date': '2024-11-29_18-01-32', 'timestamp': 1732903292, 'time_this_iter_s': 1.9912559986114502, 'time_total_s': 129.86767268180847, 'pid': 33553, 'hostname': 'user-116-99.svr-vpn-0.vpn.cl.cam.ac.uk', 'node_ip': '127.0.0.1', 'config': {'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'placement_strategy': 'PACK', 'num_gpus': 0, '_fake_gpus': False, 'num_cpus_for_main_process': 1, 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU'

KeyboardInterrupt: 

In [2]:
# read npz file in /Users/sidharrthnagappan/Downloads
import numpy as np

data = np.load("/Users/sidharrthnagappan/Downloads/305_P.npz")
print(data['data'])

[]


In [3]:
import pandas as pd

# read csv, divided by ;
data = pd.read_csv("/Users/sidharrthnagappan/Downloads/452_OpenSMILE2.3.0_mfcc.csv", sep=';')

In [4]:
data

,name,frameTime,pcm_fftMag_mfcc[0],pcm_fftMag_mfcc[1],pcm_fftMag_mfcc[2],pcm_fftMag_mfcc[3],pcm_fftMag_mfcc[4],pcm_fftMag_mfcc[5],pcm_fftMag_mfcc[6],pcm_fftMag_mfcc[7],...,pcm_fftMag_mfcc_de_de[3],pcm_fftMag_mfcc_de_de[4],pcm_fftMag_mfcc_de_de[5],pcm_fftMag_mfcc_de_de[6],pcm_fftMag_mfcc_de_de[7],pcm_fftMag_mfcc_de_de[8],pcm_fftMag_mfcc_de_de[9],pcm_fftMag_mfcc_de_de[10],pcm_fftMag_mfcc_de_de[11],pcm_fftMag_mfcc_de_de[12]
0,'unknown',0.00,-9.915971,-8.620132,2.039084,-5.163440,2.006905,18.294490,22.194500,15.895390,...,0.152880,0.489295,0.710587,0.540104,0.130175,0.031273,-0.916245,-1.907472,-0.958420,-0.135700
1,'unknown',0.01,-6.664284,-4.194248,-4.648168,-3.684016,-4.954080,-6.889012,7.564898,-0.844071,...,-0.203163,0.871625,2.274281,1.531581,1.520242,0.048998,-1.585407,-2.866117,-1.770200,0.156029
2,'unknown',0.02,-6.332829,0.685269,-0.365634,2.003712,0.390207,-0.603974,-1.424609,2.812038,...,-0.900279,0.538887,2.665507,2.607594,2.207535,-0.088446,-1.505144,-1.711007,-1.432200,0.416258
3,'unknown',0.03,-9.073243,0.205635,0.152932,1.457144,0.118186,-5.632328,2.595091,-9.695595,...,-0.727857,0.464294,2.244686,2.481748,2.309757,-0.095080,-0.493684,0.328701,-0.890489,0.639589
4,'unknown',0.04,-5.547980,3.337821,-0.236531,2.846203,5.405900,4.883540,7.485616,7.462225,...,-0.057012,0.143035,0.048417,1.076983,0.554909,0.119702,2.025989,2.261990,-0.292087,0.302947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88903,'unknown',889.03,6.596205,9.779520,17.572750,19.726100,15.150250,12.101650,25.052470,20.088950,...,-1.633085,-2.086792,-1.583521,-0.576014,0.486673,0.896709,1.213446,2.731232,2.818074,-0.922672
88904,'unknown',889.04,-1.099818,5.529945,11.248440,15.844810,12.723560,4.295323,20.629760,10.558960,...,-1.572607,-0.766718,-1.150338,-1.013006,-0.534065,-1.417233,-1.234026,0.508356,2.143103,0.315381
88905,'unknown',889.05,-4.075032,2.090782,7.954472,12.467060,8.319486,3.370588,28.262950,10.999320,...,-0.499376,0.946104,-0.529290,-1.563444,-1.549245,-3.077513,-1.540916,-0.110525,1.013761,1.093440
88906,'unknown',889.06,-4.853426,-5.538296,4.713272,5.192731,-3.556020,-0.729859,30.653840,22.930960,...,0.742765,1.914226,0.516001,-1.311308,-1.343467,-1.682199,-1.308228,-1.364408,-0.566124,0.827624
